In [1]:
#########################################################################################################
############# IN THIS NOTEBOOK, A FULL PROCESS EXAMPLE IS MADE TO DEMNOSTRATE THE STEPS #################
#########################################################################################################

In [2]:
import tensorflow as tf
import numpy as np
import solver
import utils
import cv2

IMG_PATH = 'example.png'

# Load saved model
model = tf.keras.models.load_model("../app/"+utils.MODEL_PATH)

In [3]:

#####################################################  DETECTION  ####################################################

# Prepare image for character detection
image = cv2.imread(IMG_PATH)
thresh = utils.preproc(image)
cnts = utils.findContours(thresh)


chars = {}
for c in cnts:

    # Remove minor distractions by setting a minimal area size
    if cv2.contourArea(c) > 75:

        # Extract ROI and starting position on x axis
        roi,x = utils.extractChar(thresh,c,position=True)
        chars[x] = utils.scaleAndPad(roi)

# Sort by keys (starting positions on the x axis) to read the characters left to right
sorted_keys = sorted(chars)

In [4]:

#####################################################  RECOGNITION  ##################################################

equation = []
probabilities = []
for key in sorted_keys:

    char = chars[key]
    # Model expects a batch of images, (batch_size, img_width, img_height, img_channels)
    # Since this is a single image, batch_size = 1, it's square so img_width = img_height and it's B&W so img_channels = 1)
    char = char.reshape(1,utils.IMG_SIZE,utils.IMG_SIZE,1)
    # Get the most probable prediction from the model
    prediction = model.predict(char)
    class_no = np.argmax(prediction)

    # Check for probability. If it isn't high enough, discard the prediction
    probability = prediction[:,class_no][0]
    if probability < utils.PROB_TH:
        continue
    probabilities.append(probability)

    # Decode the prediction into class name
    char = utils.CLASS_NAMES[class_no]
    equation.append(char)


print("predicted classes:",equation)
print("probabilities:",probabilities)

predicted classes: ['2', 'minus', '5']
probabilities: [1.0, 0.99999964, 0.9999683]


In [5]:

#####################################################  PARSING AND SOLVING  ###########################################
eq = solver.predictionsToEquation(equation)
print("Equation:",solver.preparePrint(eq))

eq_unary = solver.unaryMinuses(eq)
print("Chunked into elements:",eq_unary)

postfix = solver.shuntingYard(eq_unary)
print("Postfix notation:",postfix)

solution = solver.solvePostfix(postfix)
print("Solution:",solver.preparePrint(solution))


Equation: 2-5
Chunked into elements: ['2', '-', '5']
Postfix notation: ['2', '5', '-']
Solution: -3
